<a href="https://colab.research.google.com/github/sagar4real/CMP7005-PRAC1/blob/main/01_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
print("Libraries imported successfully")

Libraries imported successfully


In [6]:
import glob
import os

# Folder where your CSVs are stored
DATA_FOLDER = "/content/CMP7005-PRAC1/data"

# Pattern to match your files
pattern = os.path.join(DATA_FOLDER, "*_data.csv")

# List all matching files
csv_files = sorted(glob.glob(pattern))
print("Files detected:", csv_files)

if len(csv_files) == 0:
    raise FileNotFoundError("No files found !")

# Load and append
df_list = []
for file in csv_files:
    temp = pd.read_csv(file)
    df_list.append(temp)

# Merge all into one DataFrame
merged_df = pd.concat(df_list, ignore_index=True)

# Save merged output
output_path = os.path.join(DATA_FOLDER, "merged_air_quality_data.csv")
merged_df.to_csv(output_path, index=False)

print("Merged file saved to:", output_path)
print("Final shape:", merged_df.shape)

# Display first rows
merged_df.head()


Files detected: ['/content/CMP7005-PRAC1/data/Ahmedabad_data.csv', '/content/CMP7005-PRAC1/data/Aizawl_data.csv', '/content/CMP7005-PRAC1/data/Amaravati_data.csv', '/content/CMP7005-PRAC1/data/Amritsar_data.csv', '/content/CMP7005-PRAC1/data/Bengaluru_data.csv', '/content/CMP7005-PRAC1/data/Bhopal_data.csv', '/content/CMP7005-PRAC1/data/Brajrajnagar_data.csv', '/content/CMP7005-PRAC1/data/Chandigarh_data.csv', '/content/CMP7005-PRAC1/data/Chennai_data.csv', '/content/CMP7005-PRAC1/data/Coimbatore_data.csv', '/content/CMP7005-PRAC1/data/Delhi_data.csv', '/content/CMP7005-PRAC1/data/Ernakulam_data.csv', '/content/CMP7005-PRAC1/data/Gurugram_data.csv', '/content/CMP7005-PRAC1/data/Guwahati_data.csv', '/content/CMP7005-PRAC1/data/Hyderabad_data.csv', '/content/CMP7005-PRAC1/data/Jaipur_data.csv', '/content/CMP7005-PRAC1/data/Jorapokhar_data.csv', '/content/CMP7005-PRAC1/data/Kochi_data.csv', '/content/CMP7005-PRAC1/data/Kolkata_data.csv', '/content/CMP7005-PRAC1/data/Lucknow_data.csv', '/c

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,01/01/2015,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,02/01/2015,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,03/01/2015,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,04/01/2015,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,05/01/2015,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN
